### Coleta e persistência dos dados neste notebook

**tipo de ingestão**: full load (recomendado para ingestão de dados inicial no datalake)

**origem**: Yahoo finance, Wikipedia, Fear and Greed (alternative.me)

**destino**: bronze/database/bitcoin/raw/btc.csv

**formato do data no data lake**: .csv


#### Instruções

1. Renomear o arquivo .env_exemplo para somente .env
2. Adicionar popular as variaveis conforme o padrão de nomenclatura que voce utilizar

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError
import pandas as pd
from transformers import pipeline
import yfinance as yf
import mwclient
import time
import requests
import json
from datetime import datetime
from statistics import mean
from io import StringIO
from dotenv import load_dotenv
import os

##carrega variaveis de ambiente
load_dotenv()

Extrai os dados de bitcoin do Yahoo Finance

In [3]:
def extract_btc(data_inicio: datetime) -> pd.DataFrame:
    ticker = yf.Ticker("BTC-USD")
    btc = ticker.history(start=data_inicio)
    return btc

Formata a base de bitcoin

In [4]:
def format_base(df:pd.DataFrame) -> pd.DataFrame:
    df.index = pd.to_datetime(df.index).tz_localize(None)
    del df["Dividends"]
    del df["Stock Splits"]
    df.columns = [c.lower() for c in df.columns]
    return df

In [ ]:
data_inicio = datetime.strptime('2018-02-01', '%Y-%m-%d')
btc = format_base(extract_btc(data_inicio))
btc.head()

Extrai os dados das reviews de bitcoin da wikipedia

In [6]:
def extract_reviews() -> list:
   site = mwclient.Site("en.wikipedia.org")
   page = site.pages["Bitcoin"]
   revs = list(page.revisions(start=data_inicio, dir="newer"))
   revs = sorted(revs, key=lambda rev: rev["timestamp"])
   return revs

Classifica o sentimento relacionado as reviews

seleciona o modelo utilizado para classificação das reviews

In [ ]:
sentiment_pipeline = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english")

transforma o score das classificações negativas em valores negativos

In [8]:
def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

cria a base de sentimentos

In [9]:
edits = {}
revs = extract_reviews()

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

prepara e limpa a base

In [10]:
for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [11]:
def create_edits_df(edits: dict) -> pd.DataFrame:
    edits_df = pd.DataFrame.from_dict(edits, orient="index")
    edits_df.index = pd.to_datetime(edits_df.index)
    dates = pd.date_range(start=data_inicio, end=datetime.today())
    edits_df = edits_df.reindex(dates, fill_value=0)
    rolling_edits = edits_df.rolling(30, min_periods=30).mean()
    rolling_edits = rolling_edits.dropna()
    return rolling_edits

In [ ]:
rolling_edits = create_edits_df(edits)
rolling_edits.head()

Une as bases de bitcoin e sentimentos

In [ ]:
btc = btc.merge(rolling_edits, left_index=True, right_index=True)
btc.head()

Adiciona a coluna Target e limpa a base

In [ ]:
def target_and_clean(df: pd.DataFrame) -> pd.DataFrame:
    df["tomorrow"] = df["close"].shift(-1)
    df["target"] = (df["tomorrow"] > df["close"]).astype(int)
    df.dropna(inplace=True)
    return df

btc =  target_and_clean(btc)
btc.head()

Cria colunas adicionais dataframe existente para verificar possíveis tendências em horizontes de tempo

In [15]:
def trends_col(df: pd.DataFrame) -> pd.DataFrame:
    horizons = [2, 7, 30, 60, 365]

    for horizon in horizons:
        rolling_averages = df.rolling(horizon, min_periods=1).mean()

        ratio_column = f'close_ratio_{horizon}'
        df[ratio_column] = df['close'] / rolling_averages['close']

        edit_column = f'edit_{horizon}'
        df[edit_column] = rolling_averages['edit_count']

        rolling = df.rolling(horizon, closed='left', min_periods=1).mean()
        trend_column = f'trend_{horizon}'
        df[trend_column] = rolling['target']

    return df

btc = trends_col(btc)

Extrai a base de Fear and Greed da Alternative.me

In [35]:
def collect_fear_greed(api_url:str, limit:int, drop_colls:list, rename_colls:list) -> pd.DataFrame:
    response = requests.get(f"{api_url}?limit={limit}")
    n_data = response.json()
    df = pd.read_json(StringIO(json.dumps(n_data["data"])))
    df.drop(columns=drop_colls, axis=1, inplace=True)
    df.rename(columns=rename_colls, inplace=True)
    df.set_index("date", inplace=True)
    return df

In [36]:
api_url = "https://api.alternative.me/fng/"
colls_to_drop = ["time_until_update"]
coll_names = {"value":"fng_index", 
              "value_classification":"fng_class",
              "timestamp": "date"}

data_fg = collect_fear_greed(api_url=api_url, limit=0, drop_colls=colls_to_drop, rename_colls=coll_names)

In [33]:
data_fg = data_fg.sort_index()

Une as bases de bitcoin, sentimentos e fear and greed

In [ ]:
btc = btc.merge(data_fg, left_index=True, right_index=True)
btc.head()

SUGESTÃO: se for trabalhar nos dados a partir deste ponto, descomente a celula abaixo para gerar um backup do dataframe para agilizar seu trabalho

In [20]:
# work_backup = btc.copy()
# work_backup.head()

Prepara Colunas adicionais (Reavaliar após a adição de novas features de Trend)

In [21]:
# def improve_df(df: pd.DataFrame) -> pd.DataFrame:
#     df["fng_in_yest"] = df["fng_index"].shift(1)
#     df["fng_class_yest"] = df["fng_class"].shift(1)
#     df["edit_count_y"] = df["edit_count"].shift(1)
#     df["sentiment_y"] = df["sentiment"].shift(1)
#     df["neg_sentiment_y"] = df["neg_sentiment"].shift(1)
#     df.dropna(inplace=True)
#     return df

# btc = improve_df(btc)

Organiza as colunas

In [ ]:
sequencia = [
       'open', 'high', 'low', 'close', 'volume', 'edit_count', 'sentiment',
       'neg_sentiment', 'tomorrow', 'close_ratio_2', 'edit_2',
       'trend_2', 'close_ratio_7', 'edit_7', 'trend_7', 'close_ratio_30',
       'edit_30', 'trend_30', 'close_ratio_60', 'edit_60', 'trend_60',
       'close_ratio_365', 'edit_365', 'trend_365', 'fng_index', 'fng_class',
       'target'
]
btc = btc[sequencia]
btc.head()

Realiza a ingestão do dataframe no data lake

In [23]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [24]:
# Converte o dataframe em uma string CSV
csv_buffer = StringIO()
btc.to_csv(csv_buffer, index=True)

In [25]:
bucket_name = os.environ['BUCKET_NAME']
csv_file_name = os.environ['CSV_FILE_NAME']
bucket_layer = os.environ['BUCKET_LAYER']
object_name = f'{bucket_layer}/{csv_file_name}'

In [ ]:
def upload_to_s3(bucket_name, object_name, csv_buffer):
    try:
        s3_resource = boto3.resource('s3')
        req_metadata = s3_resource.Object(bucket_name, object_name).put(Body=csv_buffer.getvalue())
        status = req_metadata['ResponseMetadata']['HTTPStatusCode']
        if status == 200:
            print('Upload sucessful')
        else:
            print(f'Upload failed, status {status}')
    except ClientError as e:
        logging.error(e)

upload_to_s3(bucket_name, object_name, csv_buffer)

In [27]:
# baixa csv localmente
#btc.to_csv('btc.csv')